In [1]:
#import necessary packages
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.colors as mcolors
import datetime
from dateutil.relativedelta import relativedelta
from datetime import date
from datetime import datetime

In [2]:
#load in neccesary CSV files
diagnosis_icd9 = pd.read_csv("/project/projectdirs/m1532/Projects_MVP/_datasets/mimiciii/DIAGNOSES_ICD.csv", usecols= ["SUBJECT_ID", "ICD9_CODE"])
diag_name = pd.read_csv("/project/projectdirs/m1532/Projects_MVP/_datasets/mimiciii/D_ICD_DIAGNOSES.csv", usecols= ["ICD9_CODE", "LONG_TITLE"])
code_description = pd.read_csv("/project/projectdirs/m1532/Projects_MVP/_datasets/mimiciii/DRGCODES.csv")
patients = pd.read_csv("/project/projectdirs/m1532/Projects_MVP/_datasets/mimiciii/PATIENTS.csv", parse_dates = usecols=["SUBJECT_ID", "DOB"])
admissions = pd.read_csv("/project/projectdirs/m1532/Projects_MVP/_datasets/mimiciii/ADMISSIONS.csv", usecols=["SUBJECT_ID", "ADMITTIME"])
diagnosis_icd9.head()

,SUBJECT_ID,ICD9_CODE
0,109,40301
1,109,486
2,109,58281
3,109,5855
4,109,4254


In [3]:
#filter for only "diseases of despair" ICD9 codes
diag_dd_DF = diagnosis_icd9[diagnosis_icd9["ICD9_CODE"].str.startswith(("327", "338", "339", "346", "295", "296", "297", "298", "300", "301", "302", "307", "309", "310", "311", "313"), na = False)]
diag_dd_DF.head()

,SUBJECT_ID,ICD9_CODE
22,109,32723
24,109,33829
248,135,311
305,67,311
311,67,311


In [4]:
diag_dd_DF = pd.merge(diag_dd_DF, patients, on='SUBJECT_ID')
diag_dd_DF.head()

,SUBJECT_ID,ICD9_CODE,DOB
0,109,32723,2117-08-07 00:00:00
1,109,33829,2117-08-07 00:00:00
2,109,32723,2117-08-07 00:00:00
3,109,33829,2117-08-07 00:00:00
4,109,32723,2117-08-07 00:00:00


In [5]:
diag_dd_DF = pd.merge(diag_dd_DF, admissions, on="SUBJECT_ID")
diag_dd_DF.head()

,SUBJECT_ID,ICD9_CODE,DOB,ADMITTIME
0,109,32723,2117-08-07 00:00:00,2137-11-04 19:36:00
1,109,32723,2117-08-07 00:00:00,2138-04-16 17:44:00
2,109,32723,2117-08-07 00:00:00,2140-01-19 13:25:00
3,109,32723,2117-08-07 00:00:00,2140-02-02 02:13:00
4,109,32723,2117-08-07 00:00:00,2140-04-07 19:51:00


In [6]:
diag_dd_DF['DOB'].isnull().values.any()

False

In [7]:
diag_dd_DF['ADMITTIME'].isnull().values.any()

False

In [48]:
diag_dd_DF['AGE'] = (diag_dd_DF['ADMITTIME']-diag_dd_DF['DOB'])
diag_dd_DF = diag_dd_DF[diag_dd_DF['AGE'].dt.days > 365*18]

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [8]:
import sys
sys.maxsize

9223372036854775807

In [9]:
diag_dd_DF.count()

SUBJECT_ID    36382
ICD9_CODE     36382
DOB           36382
ADMITTIME     36382
dtype: int64

In [10]:
diag_dd_DF.groupby(['ICD9_CODE']).count()

,SUBJECT_ID,DOB,ADMITTIME
ICD9_CODE,,,
29502,1,1,1
29512,5,5,5
29520,8,8,8
29522,1,1,1
29524,2,2,2
...,...,...,...
34670,2,2,2
34680,38,38,38
34690,1362,1362,1362


In [11]:
#merge in separate dataframe to include the name/description of the ICD9 code 
diagDFdesc = pd.merge(diag_dd_DF, diag_name, on='ICD9_CODE')
diagDFdesc

,SUBJECT_ID,ICD9_CODE,DOB,ADMITTIME,LONG_TITLE
0,109,32723,2117-08-07 00:00:00,2137-11-04 19:36:00,Obstructive sleep apnea (adult)(pediatric)
1,109,32723,2117-08-07 00:00:00,2138-04-16 17:44:00,Obstructive sleep apnea (adult)(pediatric)
2,109,32723,2117-08-07 00:00:00,2140-01-19 13:25:00,Obstructive sleep apnea (adult)(pediatric)
3,109,32723,2117-08-07 00:00:00,2140-02-02 02:13:00,Obstructive sleep apnea (adult)(pediatric)
4,109,32723,2117-08-07 00:00:00,2140-04-07 19:51:00,Obstructive sleep apnea (adult)(pediatric)
...,...,...,...,...,...
36375,99085,34691,2091-02-01 00:00:00,2131-02-08 16:25:00,"Migraine, unspecified, with intractable migrai..."
36376,95841,33902,2111-12-01 00:00:00,2160-04-09 21:23:00,Chronic cluster headache
36377,96928,29621,2073-05-24 00:00:00,2138-06-03 12:36:00,"Major depressive affective disorder, single ep..."
36378,96928,29621,2073-05-24 00:00:00,2139-01-01 15:02:00,"Major depressive affective disorder, single ep..."


In [12]:
notes = pd.read_csv("/project/projectdirs/m1532/Projects_MVP/_datasets/mimiciii/NOTEEVENTS.csv", usecols = ['SUBJECT_ID','CATEGORY', 'TEXT'])
notes.head()

,SUBJECT_ID,CATEGORY,TEXT
0,22532,Discharge summary,Admission Date: [**2151-7-16**] Dischar...
1,13702,Discharge summary,Admission Date: [**2118-6-2**] Discharg...
2,13702,Discharge summary,Admission Date: [**2119-5-4**] D...
3,13702,Discharge summary,Admission Date: [**2124-7-21**] ...
4,26880,Discharge summary,Admission Date: [**2162-3-3**] D...


In [13]:
#merge dd dataframe with notes
icd9_dd_notes = pd.merge(diag_dd_DF, notes, on ='SUBJECT_ID')
icd9_dd_notes.head()

,SUBJECT_ID,ICD9_CODE,DOB,ADMITTIME,CATEGORY,TEXT
0,109,32723,2117-08-07 00:00:00,2137-11-04 19:36:00,Discharge summary,Admission Date: [**2142-5-15**] ...
1,109,32723,2117-08-07 00:00:00,2137-11-04 19:36:00,Discharge summary,Admission Date: [**2142-5-20**] ...
2,109,32723,2117-08-07 00:00:00,2137-11-04 19:36:00,Discharge summary,Admission Date: [**2142-6-18**] ...
3,109,32723,2117-08-07 00:00:00,2137-11-04 19:36:00,Discharge summary,Admission Date: [**2142-7-3**] D...
4,109,32723,2117-08-07 00:00:00,2137-11-04 19:36:00,Discharge summary,Admission Date: [**2142-7-7**] D...


In [14]:
#how many entries?
icd9_dd_notes.count()

SUBJECT_ID    8595228
ICD9_CODE     8595228
DOB           8595228
ADMITTIME     8595228
CATEGORY      8595228
TEXT          8595228
dtype: int64

In [15]:
#drop any duplicate TEXT entries 
icd9_notes_unique = icd9_dd_notes.drop_duplicates(subset='TEXT', keep='first')
icd9_notes_unique

,SUBJECT_ID,ICD9_CODE,DOB,ADMITTIME,CATEGORY,TEXT
0,109,32723,2117-08-07 00:00:00,2137-11-04 19:36:00,Discharge summary,Admission Date: [**2142-5-15**] ...
1,109,32723,2117-08-07 00:00:00,2137-11-04 19:36:00,Discharge summary,Admission Date: [**2142-5-20**] ...
2,109,32723,2117-08-07 00:00:00,2137-11-04 19:36:00,Discharge summary,Admission Date: [**2142-6-18**] ...
3,109,32723,2117-08-07 00:00:00,2137-11-04 19:36:00,Discharge summary,Admission Date: [**2142-7-3**] D...
4,109,32723,2117-08-07 00:00:00,2137-11-04 19:36:00,Discharge summary,Admission Date: [**2142-7-7**] D...
...,...,...,...,...,...,...
8595205,97492,34680,2126-09-04 00:00:00,2158-07-31 22:29:00,Nursing,TITLE:\n Pt is 31 y/o woman with history of ...
8595206,97492,34680,2126-09-04 00:00:00,2158-07-31 22:29:00,Physician,SICU\n HPI:\n 31 yo RH woman with a PMH de...
8595207,97492,34680,2126-09-04 00:00:00,2158-07-31 22:29:00,Radiology,[**2158-8-1**] 10:43 AM\n MR HEAD W/O CONTRAST...
8595208,97492,34680,2126-09-04 00:00:00,2158-07-31 22:29:00,Radiology,[**2158-7-31**] 8:52 PM\n CTA HEAD W&W/O C & R...


In [16]:
#after removing duplicates, how many entries do we have?
icd9_notes_unique.count()

SUBJECT_ID    446040
ICD9_CODE     446040
DOB           446040
ADMITTIME     446040
CATEGORY      446040
TEXT          446040
dtype: int64

In [17]:
icd9_notes_unique.loc[icd9_notes_unique["SUBJECT_ID"] == 109]

,SUBJECT_ID,ICD9_CODE,DOB,ADMITTIME,CATEGORY,TEXT
0,109,32723,2117-08-07 00:00:00,2137-11-04 19:36:00,Discharge summary,Admission Date: [**2142-5-15**] ...
1,109,32723,2117-08-07 00:00:00,2137-11-04 19:36:00,Discharge summary,Admission Date: [**2142-5-20**] ...
2,109,32723,2117-08-07 00:00:00,2137-11-04 19:36:00,Discharge summary,Admission Date: [**2142-6-18**] ...
3,109,32723,2117-08-07 00:00:00,2137-11-04 19:36:00,Discharge summary,Admission Date: [**2142-7-3**] D...
4,109,32723,2117-08-07 00:00:00,2137-11-04 19:36:00,Discharge summary,Admission Date: [**2142-7-7**] D...
...,...,...,...,...,...,...
1273,109,32723,2117-08-07 00:00:00,2137-11-04 19:36:00,Physician,Chief Complaint:\n 24 Hour Events:\n - SBP...
1275,109,32723,2117-08-07 00:00:00,2137-11-04 19:36:00,Physician,Chief Complaint:\n 24 Hour Events:\n DIALYSI...
1276,109,32723,2117-08-07 00:00:00,2137-11-04 19:36:00,Physician,Chief Complaint:\n I saw and examined the pa...
1277,109,32723,2117-08-07 00:00:00,2137-11-04 19:36:00,Physician,Chief Complaint:\n I saw and examined the pa...


In [18]:
icd9_dd_notes = icd9_notes_unique

In [19]:
#how many notes per category
icd9_dd_notes.groupby(["CATEGORY"]).count().sort_values(by=['TEXT'],ascending=False)

,SUBJECT_ID,ICD9_CODE,DOB,ADMITTIME,TEXT
CATEGORY,,,,,
Radiology,138827,138827,138827,138827,138827
Nursing,76223,76223,76223,76223,76223
Nursing/other,70539,70539,70539,70539,70539
ECG,58017,58017,58017,58017,58017
Physician,53940,53940,53940,53940,53940
Discharge summary,14722,14722,14722,14722,14722
Respiratory,11899,11899,11899,11899,11899
Echo,11788,11788,11788,11788,11788
Nutrition,3448,3448,3448,3448,3448


In [20]:
#what is the largest number of entries for a patient?
#what is the least 
icd9_dd_notes.groupby(["SUBJECT_ID"]).count().sort_values(by=['TEXT'],ascending=False)

,ICD9_CODE,DOB,ADMITTIME,CATEGORY,TEXT
SUBJECT_ID,,,,,
78076,1317,1317,1317,1317,1317
99383,1228,1228,1228,1228,1228
109,1217,1217,1217,1217,1217
27427,1209,1209,1209,1209,1209
67906,1017,1017,1017,1017,1017
...,...,...,...,...,...
68353,1,1,1,1,1
98640,1,1,1,1,1
86731,1,1,1,1,1


In [21]:
#average number of notes per patient
#dont know how skewed this is (need to look at outliers)
icd9_dd_notes.groupby('SUBJECT_ID').count().mean()

ICD9_CODE    52.309136
DOB          52.309136
ADMITTIME    52.309136
CATEGORY     52.309136
TEXT         52.309136
dtype: float64

In [22]:
#how many characters?
icd9_dd_notes["TEXT"].str.len()

0          10479
1          16179
2          11058
3          10613
4           9850
           ...  
8595205     1627
8595206     4672
8595207     1493
8595208     2975
8595209     1446
Name: TEXT, Length: 446040, dtype: int64

In [23]:
#summary of the notes
#what is the std?
icd9_dd_notes["TEXT"].str.len().describe()

count    446040.000000
mean       2435.212030
std        3046.289173
min           3.000000
25%         761.000000
50%        1360.000000
75%        2716.000000
max       50663.000000
Name: TEXT, dtype: float64

In [24]:
#average number of characters per text
icd9_dd_notes["TEXT"].str.len().mean()

2435.212030311183

In [25]:
# Load the regular expression library
import re

In [26]:
# Remove punctuation and anything that isnt a character or number 
#this process took approximately (start 1:31:16 - finish 1:33:10) 2 minutes
icd9_dd_notes['text_processed'] = icd9_dd_notes['TEXT'].map(lambda x: re.sub('[^A-Za-z0-9, ]+', '', x))

In [27]:
# Convert to lowercase
icd9_dd_notes['text_processed'] = icd9_dd_notes['text_processed'].map(lambda x: x.lower())

In [28]:
# Print out the first rows of papers to ensure re working properly
icd9_dd_notes['text_processed'].head()

0    admission date  2142515              discharge...
1    admission date  2142520              discharge...
2    admission date  2142618              discharge...
3    admission date  214273              discharge ...
4    admission date  214277              discharge ...
Name: text_processed, dtype: object

In [29]:
icd9_dd_notes.head()

,SUBJECT_ID,ICD9_CODE,DOB,ADMITTIME,CATEGORY,TEXT,text_processed
0,109,32723,2117-08-07 00:00:00,2137-11-04 19:36:00,Discharge summary,Admission Date: [**2142-5-15**] ...,admission date 2142515 discharge...
1,109,32723,2117-08-07 00:00:00,2137-11-04 19:36:00,Discharge summary,Admission Date: [**2142-5-20**] ...,admission date 2142520 discharge...
2,109,32723,2117-08-07 00:00:00,2137-11-04 19:36:00,Discharge summary,Admission Date: [**2142-6-18**] ...,admission date 2142618 discharge...
3,109,32723,2117-08-07 00:00:00,2137-11-04 19:36:00,Discharge summary,Admission Date: [**2142-7-3**] D...,admission date 214273 discharge ...
4,109,32723,2117-08-07 00:00:00,2137-11-04 19:36:00,Discharge summary,Admission Date: [**2142-7-7**] D...,admission date 214277 discharge ...


In [30]:
#adding additional index column for calling
index = tuple(range(0, 446040, 1))
icd9_dd_notes["index"] = index
icd9_dd_notes

,SUBJECT_ID,ICD9_CODE,DOB,ADMITTIME,CATEGORY,TEXT,text_processed,index
0,109,32723,2117-08-07 00:00:00,2137-11-04 19:36:00,Discharge summary,Admission Date: [**2142-5-15**] ...,admission date 2142515 discharge...,0
1,109,32723,2117-08-07 00:00:00,2137-11-04 19:36:00,Discharge summary,Admission Date: [**2142-5-20**] ...,admission date 2142520 discharge...,1
2,109,32723,2117-08-07 00:00:00,2137-11-04 19:36:00,Discharge summary,Admission Date: [**2142-6-18**] ...,admission date 2142618 discharge...,2
3,109,32723,2117-08-07 00:00:00,2137-11-04 19:36:00,Discharge summary,Admission Date: [**2142-7-3**] D...,admission date 214273 discharge ...,3
4,109,32723,2117-08-07 00:00:00,2137-11-04 19:36:00,Discharge summary,Admission Date: [**2142-7-7**] D...,admission date 214277 discharge ...,4
...,...,...,...,...,...,...,...,...
8595205,97492,34680,2126-09-04 00:00:00,2158-07-31 22:29:00,Nursing,TITLE:\n Pt is 31 y/o woman with history of ...,title pt is 31 yo woman with history of head...,446035
8595206,97492,34680,2126-09-04 00:00:00,2158-07-31 22:29:00,Physician,SICU\n HPI:\n 31 yo RH woman with a PMH de...,sicu hpi 31 yo rh woman with a pmh depress...,446036
8595207,97492,34680,2126-09-04 00:00:00,2158-07-31 22:29:00,Radiology,[**2158-8-1**] 10:43 AM\n MR HEAD W/O CONTRAST...,215881 1043 am mr head wo contrast ...,446037
8595208,97492,34680,2126-09-04 00:00:00,2158-07-31 22:29:00,Radiology,[**2158-7-31**] 8:52 PM\n CTA HEAD W&W/O C & R...,2158731 852 pm cta head wwo c recons cta neck...,446038


In [31]:
data_text = icd9_dd_notes[['text_processed','index']]
data_text

,text_processed,index
0,admission date 2142515 discharge...,0
1,admission date 2142520 discharge...,1
2,admission date 2142618 discharge...,2
3,admission date 214273 discharge ...,3
4,admission date 214277 discharge ...,4
...,...,...
8595205,title pt is 31 yo woman with history of head...,446035
8595206,sicu hpi 31 yo rh woman with a pmh depress...,446036
8595207,215881 1043 am mr head wo contrast ...,446037
8595208,2158731 852 pm cta head wwo c recons cta neck...,446038


In [32]:
documents = data_text

In [33]:
#import necessary packages for further word processing
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2020)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /global/homes/d/dsmorrow/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [34]:
#function to perform lemmatize and stem preprocessing steps on the data set.
def lemmatize_stemming(text):
    stemmer = PorterStemmer()
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [35]:
#test the stemmer and lem on one example text
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['npn', 'micu', '7', 'westallergies', 'demeroltertracyclinefull', 'codesignificant', 'events', 'able', 'to', 'wean', 'vent', 'from', 'ac', 'to', 'ps', 'settings,', 'able', 'to', 'wean', 'sedation', 'today,', 'ekg', 'doneneuro', 'pt', 'is', 'sedated', 'on', 'fentanyl', '100mcgkghr', 'from', '300', 'mghr', 'and', 'versed', '5', 'mcghr', 'from', '96', 'mcghr', 'propofol', 'continues', '', '40', 'mcgkgmin', 'pt', 'neuro', 'status', 'has', 'remianed', 'unchanged', 'since', 'yesterday', 'arousable', 'to', 'voice', 'and', 'stimuli,', 'moves', 'all', 'four', 'extremeties', 'on', 'bed,', 'weak', 'gag,', 'cough', 'reflex', 'intact,', 'pupils', '3mm', '', ',', 'follows', 'simple', 'commands', 'inconsistently', 'haldol', 'given', 'as', 'orderedresp', 'pt', 'is', 'trached', 'received', 'on', 'vent', 'settings', 'ac50tv350rr28peep8', 'abg', 'wnl,', 'able', 'to', 'wean', 'vent', 'settings', 'to', 'cpap', '', 'ps', '50peep8ps20', 'rr1215', 'abg', 'wnl,', 'ls', 'clear,', 'cxr', 'bil

In [36]:
#remove any na that would cause problems in the lda model
documents = documents.dropna(subset=['text_processed'])

In [37]:
doc_group1 = documents.iloc[0:1000]
doc_group1.head()

,text_processed,index
0,admission date 2142515 discharge...,0
1,admission date 2142520 discharge...,1
2,admission date 2142618 discharge...,2
3,admission date 214273 discharge ...,3
4,admission date 214277 discharge ...,4


In [38]:
#use preprocess function on group1
processed_docs = doc_group1['text_processed'].map(preprocess)
processed_docs

0       [admiss, date, discharg, date, date, birth, fs...
1       [admiss, date, discharg, date, date, birth, fs...
2       [admiss, date, discharg, date, date, birth, fs...
3       [admiss, date, discharg, date, date, birth, fs...
4       [admiss, date, discharg, date, date, birth, fs...
                              ...                        
1055    [micu, bilat, vein, clip, clip, number, radiol...
1056    [conrast, right, clip, clip, number, radiolog,...
1057    [conrast, right, clip, clip, number, radiolog,...
1058    [chest, portabl, clip, clip, number, radiolog,...
1059    [chest, portabl, clip, clip, number, radiolog,...
Name: text_processed, Length: 1000, dtype: object

In [39]:
#create dictionary of words and number of appearances 
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 abdomin
1 abdominalpain
2 abl
3 abus
4 accid
5 acut
6 aday
7 admiss
8 admit
9 admittedd
10 afebril


In [40]:
#filter out words that appear in less than (15) documents
#only keep the first 10000
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [41]:
#go through each document and report words and occurrences using doc2box for token id and amount
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [42]:
#identify what the word is based on token and number of appearances
#765 is a sample index from documents
bow_doc_765 = bow_corpus[765]
for i in range(len(bow_doc_765)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_765[i][0], 
dictionary[bow_doc_765[i][0]], 
bow_doc_765[i][1]))

Word 3 ("acut") appears 1 time.
Word 27 ("blood") appears 1 time.
Word 29 ("bridg") appears 1 time.
Word 33 ("check") appears 1 time.
Word 34 ("chief") appears 1 time.
Word 37 ("chronic") appears 1 time.
Word 48 ("coumadin") appears 1 time.
Word 53 ("crisi") appears 1 time.
Word 71 ("drip") appears 1 time.
Word 75 ("emerg") appears 1 time.
Word 83 ("event") appears 1 time.
Word 94 ("glucos") appears 1 time.
Word 106 ("hour") appears 2 time.
Word 113 ("includ") appears 1 time.
Word 125 ("labetalol") appears 1 time.
Word 130 ("line") appears 2 time.
Word 136 ("med") appears 1 time.
Word 149 ("nausea") appears 2 time.
Word 155 ("nifedipin") appears 1 time.
Word 157 ("note") appears 3 time.
Word 165 ("patch") appears 2 time.
Word 167 ("percocet") appears 1 time.
Word 174 ("prednison") appears 1 time.
Word 180 ("provid") appears 1 time.
Word 186 ("rebound") appears 1 time.
Word 196 ("restart") appears 1 time.
Word 199 ("review") appears 2 time.
Word 211 ("sign") appears 1 time.
Word 213 ("s

In [43]:
#determine the TF-IDF scores or weight of a word within a document
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.0173180133591342),
 (1, 0.037726875857839993),
 (2, 0.04075002636008019),
 (3, 0.010907674361755217),
 (4, 0.037726875857839993),
 (5, 0.03132641714436169),
 (6, 0.026039845803148737),
 (7, 0.023230504337241494),
 (8, 0.015513443921819928),
 (9, 0.010556721486522319),
 (10, 0.03912465330789028),
 (11, 0.032445604479549146),
 (12, 0.04508099522676178),
 (13, 0.02254049761338089),
 (14, 0.01580057744264134),
 (15, 0.020101427365367085),
 (16, 0.03297210133772002),
 (17, 0.02145823941183696),
 (18, 0.037726875857839993),
 (19, 0.02273283297721263),
 (20, 0.0383507896082012),
 (21, 0.023490935819000882),
 (22, 0.04200179930932068),
 (23, 0.021205032705606797),
 (24, 0.023647278419810763),
 (25, 0.0338177708255089),
 (26, 0.033008929977104834),
 (27, 0.031589894646032186),
 (28, 0.041356341125407196),
 (29, 0.0247721854308978),
 (30, 0.010134670226441512),
 (31, 0.031239188209780444),
 (32, 0.04075002636008019),
 (33, 0.017662914112741016),
 (34, 0.009836106840739574),
 (35, 0.057654

In [44]:
#train lda model using only bow_corpus
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.015*"continu" + 0.014*"mgdl" + 0.010*"give" + 0.009*"dose" + 0.009*"meql" + 0.009*"today" + 0.007*"follow" + 0.007*"home" + 0.007*"start" + 0.006*"stool"
Topic: 1 
Words: 0.016*"leav" + 0.011*"normal" + 0.010*"right" + 0.007*"rhythm" + 0.007*"continu" + 0.007*"ventricular" + 0.007*"tablet" + 0.006*"daili" + 0.006*"trace" + 0.006*"give"
Topic: 2 
Words: 0.037*"tablet" + 0.014*"daili" + 0.013*"blood" + 0.012*"patch" + 0.011*"leav" + 0.010*"neg" + 0.008*"continu" + 0.008*"time" + 0.008*"releas" + 0.007*"discharg"
Topic: 3 
Words: 0.014*"continu" + 0.011*"give" + 0.009*"follow" + 0.008*"action" + 0.008*"med" + 0.008*"respons" + 0.007*"dilaudid" + 0.007*"leav" + 0.007*"chronic" + 0.007*"home"
Topic: 4 
Words: 0.014*"leav" + 0.011*"cathet" + 0.009*"right" + 0.009*"hospit" + 0.007*"continu" + 0.007*"clip" + 0.006*"chest" + 0.006*"tablet" + 0.006*"reason" + 0.006*"normal"
Topic: 5 
Words: 0.016*"respons" + 0.014*"action" + 0.012*"med" + 0.011*"receiv" + 0.011*"continu" + 0.0

In [45]:
# train lda model using tf-idf word weights already established
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.010*"action" + 0.010*"respons" + 0.005*"date" + 0.004*"dilaudid" + 0.004*"shift" + 0.004*"cont" + 0.004*"continu" + 0.004*"valuabl" + 0.004*"med" + 0.004*"order"
Topic: 1 Word: 0.012*"tablet" + 0.006*"mgdl" + 0.005*"neg" + 0.005*"leav" + 0.004*"puls" + 0.004*"meql" + 0.004*"continu" + 0.004*"daili" + 0.004*"cathet" + 0.003*"ventricular"
Topic: 2 Word: 0.026*"action" + 0.024*"respons" + 0.015*"signific" + 0.013*"trace" + 0.010*"ventricular" + 0.010*"compar" + 0.009*"chronic" + 0.009*"previou" + 0.008*"wave" + 0.008*"failur"
Topic: 3 Word: 0.010*"mgdl" + 0.006*"meql" + 0.005*"continu" + 0.005*"puls" + 0.004*"comment" + 0.004*"balanc" + 0.004*"hand" + 0.004*"trace" + 0.003*"total" + 0.003*"dyspnea"
Topic: 4 Word: 0.012*"mgdl" + 0.008*"meql" + 0.007*"respons" + 0.006*"action" + 0.006*"cont" + 0.005*"labetolol" + 0.005*"puls" + 0.004*"stage" + 0.004*"heparin" + 0.004*"total"
Topic: 5 Word: 0.010*"mgdl" + 0.008*"meql" + 0.007*"puls" + 0.005*"normal" + 0.005*"sound" + 0.004*"

In [46]:
## determine which topic a certain document belongs to
#again 765 is a sample index
for index, score in sorted(lda_model[bow_corpus[765]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.993998110294342	 
Topic: 0.015*"mgdl" + 0.011*"continu" + 0.011*"meql" + 0.009*"total" + 0.008*"puls" + 0.008*"balanc" + 0.008*"leav" + 0.008*"lab" + 0.007*"code" + 0.007*"system"
